In [4]:
import pandas as pd
import polars as pl
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [5]:
df_new = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_train.tsv', delimiter = '\t')

In [6]:
df_new.dtypes

Sentence_id     int64
Text           object
class_label    object
dtype: object

In [7]:
print(df_new[df_new['class_label'] == 'No'].count()) # gold_data -> 253
print(df_new[df_new['class_label'] == 'Yes'].count()) # gold_data -> 88

Sentence_id    17088
Text           17088
class_label    17088
dtype: int64
Sentence_id    5413
Text           5413
class_label    5413
dtype: int64


In [8]:
# class FeatureEngineering:
#     """ cleane_text : take text column and reaturn tokens
#     """
   
#     # def class_label_to_numeric()
      

In [9]:
df_new.head()

,Sentence_id,Text,class_label
0,30313,And so I know that this campaign has caused so...,No
1,19099,"Now, let's balance the budget and protect Medi...",No
2,33964,I'd like to mention one thing.,No
3,16871,I must remind him the Democrats have controlle...,Yes
4,13150,And to take a chance uh - now be - and not mak...,No


In [1]:
# for index, row in df_new.iterrows():
#     print(row['Text'])
#     print(row['class_label'])

In [98]:
df_last = pd.read_csv('add_embedding_features_9.csv')

In [99]:
df_last.head(2)

,Sentence_id,Text,class_label,Tokens,text_length,sentiment_score_veda,sentiment_score_textblob,sentiment_score_bert,sentiment_score_roberta,bert_sent_neg,...,cleaned_text,cleaned_text_length,contains_question_mark,contains_exclamation,contains_ellipsis,num_exclamations,num_questions,num_ellipses,punctuation_count,embedding
0,30313,And so I know that this campaign has caused so...,No,"['campaign', 'caused', 'questioning', 'worries...",118,-0.4939,0.5,"[0.9687058329582214, 0.031294114887714386]","[0.11130036413669586, 0.014399373903870583, 0....",0.968706,...,And so I know that this campaign has caused so...,117,False,False,False,0,0,0,1,[[-2.65752003e-02 2.48833038e-02 -1.60991415e...
1,19099,"Now, let's balance the budget and protect Medi...",No,"['lets', 'balance', 'budget', 'protect', 'medi...",92,0.3818,0.0,"[0.5200970768928528, 0.4799029529094696]","[0.10983521491289139, 0.03143635019659996, 0.7...",0.520097,...,Now lets balance the budget and protect Medica...,87,False,False,False,0,0,0,5,[[-6.90975189e-02 -5.70267588e-02 -1.18759215e...


# 1. yes\no -> 1\0

In [102]:
# new label from text to 0/1
df_new['label'] = df_new['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

## TEXT MANIPULATION- cleaned the text and create new columns

In [105]:
# clean text and then tokenize
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [106]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Token generation

In [18]:

def clean_text(text):
        # remove punctuation
        text = ''.join([char for char in text if char not in string.punctuation]) # 
        # tokenize
        tokens = word_tokenize(text)
        # remove stop words
        token = [word for word in tokens if word.lower() not in stop_words]
        return token

In [108]:
# creating new column name Token
df_new['Tokens'] = df_new['Text'].apply(clean_text)

In [19]:
#df_new.head()
# df_new.drop(['vader_neg', 'vader_neu', 'vader_pos',	'vader_compound'], axis = 1, inplace = True)

In [110]:
# nwrong feature - counting the whole length, not counting the words
# def verious_text_length(text):
#         text_length = len(text)
#         return text_length

# 3. Text Length (without cleaning)

In [111]:
df_new['raw_text_length'] = df_new['Text'].apply(verious_text_length)

# 4. cleaned text and cleaned text length

In [112]:
def cleaned_text(text):
        # lowercase the data
        lower_text = text.lower()
        # removing the punctuations
        remove_punct = ''.join([char for char in lower_text if char not in string.punctuation])
        # remove the stop words
        words = word_tokenize(remove_punct)
        remove_stop_words = [word for word in words if word not in stop_words]
        # removing the numbers
        # removing the extra space
        # replace the repetations of punctuations
        # removing emojis
        # removing emoticons
        # removing contractions
        
        return ' '.join(remove_stop_words)

In [113]:
df_new['cleaned_text'] = df_new['Text'].apply(cleaned_text)

In [114]:
df_new['cleaned_text_length'] = df_new['cleaned_text'].apply(verious_text_length)

## TEXT SENTIMENT ANALYSIS -  score and create new columns

In [115]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [116]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [117]:
sia = SentimentIntensityAnalyzer()

# 5. sentiment score vedar

In [118]:
 def get_vader_sentiment_score(text):
        return sia.polarity_scores(text)

In [119]:
df_new['sentiment_scores'] = df_new['Text'].apply(get_vader_sentiment_score) 
# extract individual sentiment into seperate columns
df_new = pd.concat([df_new.drop(['sentiment_scores'], axis = 1), 
                   df_new['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})


In [20]:
# df_new.head()

In [91]:
# 'text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
#                             'roberta_sent_pos', 'names', 'organizations', 'dates',
#                             'count_tokens', 'cleaned_text_length', 'punctuation_count'

# 6. roberta sentiment

In [122]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [123]:
def get_sentiment_features(text):
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits
    
    # Apply softmax to calculate probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    negative, neutral, positive = probs[0].tolist()
    
    # Define thresholds for "mixed" classification (adjust as needed)
    if max([positive, negative, neutral]) == positive and positive > 0.5:
        sentiment = "Positive"
    elif max([positive, negative, neutral]) == negative and negative > 0.5:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"
    
    return {
        "positive_score": positive,
        "negative_score": negative,
        "neutral_score": neutral,
        "sentiment_class": sentiment
    }

In [124]:
df_new['roberta_sentiment'] = df_new['Text'].apply(get_sentiment_features) 

In [125]:
df_new.head(2)

,Sentence_id,Text,class_label,label,Tokens,raw_text_length,cleaned_text,cleaned_text_length,vader_neg,vader_neu,vader_pos,vader_compound,roberta_sentiment
0,30313,And so I know that this campaign has caused so...,No,0,"[know, campaign, caused, questioning, worries,...",118,know campaign caused questioning worries part ...,71,0.181,0.819,0.000,-0.4939,"{'positive_score': 0.01179150864481926, 'negat..."
1,19099,"Now, let's balance the budget and protect Medi...",No,0,"[lets, balance, budget, protect, Medicare, Med...",92,lets balance budget protect medicare medicaid ...,67,0.000,0.822,0.178,0.3818,"{'positive_score': 0.2761276066303253, 'negati..."


## different score into different columns (roberta)

In [127]:
df_new["positive_score"] = df_new["roberta_sentiment"].apply(lambda x: x["positive_score"])
df_new["negative_score"] = df_new["roberta_sentiment"].apply(lambda x: x["negative_score"])
df_new["neutral_score"] = df_new["roberta_sentiment"].apply(lambda x: x["neutral_score"])
df_new["sentiment_class"] = df_new["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [135]:
df_new.head()

,Sentence_id,Text,class_label,label,Tokens,raw_text_length,cleaned_text,cleaned_text_length,vader_neg,vader_neu,...,vader_compound,roberta_sentiment,positive_score,negative_score,neutral_score,sentiment_class,is_positive,is_negative,is_mixed,token_length
0,30313,And so I know that this campaign has caused so...,No,0,"[know, campaign, caused, questioning, worries,...",118,know campaign caused questioning worries part ...,71,0.181,0.819,...,-0.4939,"{'positive_score': 0.01179150864481926, 'negat...",0.011792,0.662901,0.325308,Negative,0,1,0,10
1,19099,"Now, let's balance the budget and protect Medi...",No,0,"[lets, balance, budget, protect, Medicare, Med...",92,lets balance budget protect medicare medicaid ...,67,0.000,0.822,...,0.3818,"{'positive_score': 0.2761276066303253, 'negati...",0.276128,0.131543,0.592329,Mixed,0,0,1,8
2,33964,I'd like to mention one thing.,No,0,"[Id, like, mention, one, thing]",30,id like mention one thing,25,0.000,0.667,...,0.3612,"{'positive_score': 0.10868842899799347, 'negat...",0.108688,0.039342,0.851969,Mixed,0,0,1,5
3,16871,I must remind him the Democrats have controlle...,Yes,1,"[must, remind, Democrats, controlled, Congress...",124,must remind democrats controlled congress last...,78,0.000,1.000,...,0.0000,"{'positive_score': 0.06000441685318947, 'negat...",0.060004,0.232579,0.707417,Mixed,0,0,1,11
4,13150,And to take a chance uh - now be - and not mak...,No,0,"[take, chance, uh, make, every, effort, could,...",161,take chance uh make every effort could make pr...,93,0.146,0.687,...,0.2023,"{'positive_score': 0.02639774978160858, 'negat...",0.026398,0.688611,0.284991,Negative,0,1,0,15


## roberta sentiment class (pos neg mix)

In [129]:
# negative - 0, mixed - 1, positive - 2
df_new['sentiment_class'].unique()

array(['Negative', 'Mixed', 'Positive'], dtype=object)

# 7. create new columns for neg, pos, mix using roberta sentiment class

In [130]:
df_new["is_positive"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
df_new["is_negative"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
df_new["is_mixed"] = df_new["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)

In [131]:
# count tokens
df_new.dtypes

Sentence_id              int64
Text                    object
class_label             object
label                    int64
Tokens                  object
raw_text_length          int64
cleaned_text            object
cleaned_text_length      int64
vader_neg              float64
vader_neu              float64
vader_pos              float64
vader_compound         float64
roberta_sentiment       object
positive_score         float64
negative_score         float64
neutral_score          float64
sentiment_class         object
is_positive              int64
is_negative              int64
is_mixed                 int64
dtype: object

In [132]:
df_new['Tokens'].iloc[0]

['know',
 'campaign',
 'caused',
 'questioning',
 'worries',
 'part',
 'many',
 'leaders',
 'across',
 'globe']

# 8. Token length

In [133]:
def count_tokens(tokens):
    return len(tokens)

In [134]:
df_new['token_length'] = df_new['Tokens'].apply(count_tokens)

In [157]:
# from transformers import pipeline

In [158]:
# ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)

In [155]:
import spacy
from collections import Counter

In [156]:
nlp = spacy.load("en_core_web_sm")

In [162]:
def count_entities_to_columns(text):
    

    # # Process each row in the DataFrame
    # for text in df[text_column]:
    #     # Default counts for each row
    names = 0
    orgs = 0
    dates = 0

        # Process text using spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            names += 1
        elif ent.label_ == "ORG":
            orgs += 1
        elif ent.label_ == "DATE":
            dates += 1

    return {
        "names": names,
        "organizations": orgs,
        "dates": dates
    }

# 9. count column dictionary with names organizations, dates

In [163]:
df_new['counts'] = df_new['Text'].apply(count_entities_to_columns)

In [164]:
df_new.head()

,Sentence_id,Text,class_label,label,Tokens,raw_text_length,cleaned_text,cleaned_text_length,vader_neg,vader_neu,...,positive_score,negative_score,neutral_score,sentiment_class,is_positive,is_negative,is_mixed,token_length,punctuation_count,counts
0,30313,And so I know that this campaign has caused so...,No,0,"[know, campaign, caused, questioning, worries,...",118,know campaign caused questioning worries part ...,71,0.181,0.819,...,0.011792,0.662901,0.325308,Negative,0,1,0,10,0,"{'names': 0, 'organizations': 0, 'dates': 0}"
1,19099,"Now, let's balance the budget and protect Medi...",No,0,"[lets, balance, budget, protect, Medicare, Med...",92,lets balance budget protect medicare medicaid ...,67,0.000,0.822,...,0.276128,0.131543,0.592329,Mixed,0,0,1,8,0,"{'names': 0, 'organizations': 2, 'dates': 0}"
2,33964,I'd like to mention one thing.,No,0,"[Id, like, mention, one, thing]",30,id like mention one thing,25,0.000,0.667,...,0.108688,0.039342,0.851969,Mixed,0,0,1,5,0,"{'names': 0, 'organizations': 0, 'dates': 0}"
3,16871,I must remind him the Democrats have controlle...,Yes,1,"[must, remind, Democrats, controlled, Congress...",124,must remind democrats controlled congress last...,78,0.000,1.000,...,0.060004,0.232579,0.707417,Mixed,0,0,1,11,0,"{'names': 0, 'organizations': 1, 'dates': 0}"
4,13150,And to take a chance uh - now be - and not mak...,No,0,"[take, chance, uh, make, every, effort, could,...",161,take chance uh make every effort could make pr...,93,0.146,0.687,...,0.026398,0.688611,0.284991,Negative,0,1,0,15,0,"{'names': 0, 'organizations': 0, 'dates': 0}"


# 10. new columns seperated the count column into 3 different column for names organizations dates

In [166]:
df_new["names"] = df_new["counts"].apply(lambda x: x["names"])
df_new["organizations"] = df_new["counts"].apply(lambda x: x["organizations"])
df_new["dates"] = df_new["counts"].apply(lambda x: x["dates"])
# df_new["sentiment_class"] = df_new["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [151]:
import re

In [152]:
def count_punctuation(text):
    # Define a regular expression pattern to match punctuation characters
    punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    # Find all punctuation marks in the text
    punctuation_count = len(re.findall(punctuation_pattern, text))
    return punctuation_count

# 11. puctuatoin count column

In [153]:
df_new['punctuation_count'] = df_new['Text'].apply(count_punctuation)

In [168]:
df_new.columns

Index(['Sentence_id', 'Text', 'class_label', 'label', 'Tokens',
       'raw_text_length', 'cleaned_text', 'cleaned_text_length', 'vader_neg',
       'vader_neu', 'vader_pos', 'vader_compound', 'roberta_sentiment',
       'positive_score', 'negative_score', 'neutral_score', 'sentiment_class',
       'is_positive', 'is_negative', 'is_mixed', 'token_length',
       'punctuation_count', 'counts', 'names', 'organizations', 'dates'],
      dtype='object')

In [170]:
# 'text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
# 'roberta_sent_pos', 'names', 'organizations', 'dates',
# 'count_tokens', 'cleaned_text_length', 'punctuation_count
df_new.rename(columns = {'raw_text_length': 'text_length', 'negative_score': 'roberta_sent_neg', 'neutral_score': 'roberta_sent_mixed', 'positive_score': 'roberta_sent_pos', 'token_length': 'count_tokens'}, inplace = True)

In [171]:
df_new.head()

,Sentence_id,Text,class_label,label,Tokens,text_length,cleaned_text,cleaned_text_length,vader_neg,vader_neu,...,sentiment_class,is_positive,is_negative,is_mixed,count_tokens,punctuation_count,counts,names,organizations,dates
0,30313,And so I know that this campaign has caused so...,No,0,"[know, campaign, caused, questioning, worries,...",118,know campaign caused questioning worries part ...,71,0.181,0.819,...,Negative,0,1,0,10,0,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0
1,19099,"Now, let's balance the budget and protect Medi...",No,0,"[lets, balance, budget, protect, Medicare, Med...",92,lets balance budget protect medicare medicaid ...,67,0.000,0.822,...,Mixed,0,0,1,8,0,"{'names': 0, 'organizations': 2, 'dates': 0}",0,2,0
2,33964,I'd like to mention one thing.,No,0,"[Id, like, mention, one, thing]",30,id like mention one thing,25,0.000,0.667,...,Mixed,0,0,1,5,0,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0
3,16871,I must remind him the Democrats have controlle...,Yes,1,"[must, remind, Democrats, controlled, Congress...",124,must remind democrats controlled congress last...,78,0.000,1.000,...,Mixed,0,0,1,11,0,"{'names': 0, 'organizations': 1, 'dates': 0}",0,1,0
4,13150,And to take a chance uh - now be - and not mak...,No,0,"[take, chance, uh, make, every, effort, could,...",161,take chance uh make every effort could make pr...,93,0.146,0.687,...,Negative,0,1,0,15,0,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0


In [172]:
df_new.to_csv('07_01_data.csv', index = False)

In [8]:
df_gold = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/test_english_gold/CT24_checkworthy_english_test_gold.tsv', delimiter = '\t')

In [9]:
df_gold.head()

,Sentence_id,Text,class_label
0,35988,They said they were just going to get inspecto...,Yes
1,35991,"And from that point on, I've voted to -- I mov...",Yes
2,36029,I sit on the Senate Armed Services Committee.,No
3,36043,We need to depend on all of our tools -- diplo...,No
4,36063,And that is -- and I don't know if my colleagu...,No


In [12]:
df_gold['class_label'].unique()

array(['Yes', 'No'], dtype=object)

In [13]:
df_gold[df_gold['class_label'] == 'Yes'].count()

Sentence_id    88
Text           88
class_label    88
dtype: int64

In [14]:
df_gold[df_gold['class_label'] == 'No'].count()

Sentence_id    253
Text           253
class_label    253
dtype: int64